## Logging Info

In [3]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    threads = list()
    for index in range(3):
        logging.info("Main    : create and start thread %d.", index)
        x = threading.Thread(target=thread_function, args=(index,))
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        logging.info("Main    : before joining thread %d.", index)
        thread.join()
        logging.info("Main    : thread %d done", index)


14:47:09: Main    : create and start thread 0.
14:47:09: Thread 0: starting
14:47:09: Main    : create and start thread 1.
14:47:09: Thread 1: starting
14:47:09: Main    : create and start thread 2.
14:47:09: Thread 2: starting
14:47:09: Main    : before joining thread 0.
14:47:11: Thread 0: finishing
14:47:11: Main    : thread 0 done
14:47:11: Main    : before joining thread 1.
14:47:11: Thread 1: finishing
14:47:11: Thread 2: finishing
14:47:11: Main    : thread 1 done
14:47:11: Main    : before joining thread 2.
14:47:11: Main    : thread 2 done


In [5]:
import concurrent.futures

# [rest of code]

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(thread_function, range(3))


14:47:17: Thread 0: starting
14:47:17: Thread 1: starting
14:47:17: Thread 2: starting
14:47:19: Thread 0: finishing
14:47:19: Thread 1: finishing
14:47:19: Thread 2: finishing


## Race Condition

### Example 1

In [6]:
import threading

counter = 0
lock = threading.Lock()

def increment_counter():
    global counter
    for _ in range(1000000):
        counter += 1

# Create two threads
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)

# Start the threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

# Print the final value of the counter
print("Final Counter Value:", counter)

Final Counter Value: 1491767


### Example 2

In [8]:
class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)


In [9]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    logging.info("Testing update. Ending value is %d.", database.value)


14:47:23: Testing update. Starting value is 0.
14:47:23: Thread 0: starting update
14:47:23: Thread 1: starting update
14:47:23: Thread 0: finishing update
14:47:23: Thread 1: finishing update
14:47:23: Testing update. Ending value is 1.


In [11]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        logging.getLogger().setLevel(logging.DEBUG)
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)


In [12]:
if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    database = FakeDatabase()
    logging.info("Testing update. Starting value is %d.", database.value)
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for index in range(2):
            executor.submit(database.locked_update, index)
    logging.info("Testing update. Ending value is %d.", database.value)

14:47:52: Testing update. Starting value is 0.
14:47:52: Thread 0: starting update
14:47:52: Thread 1: starting update
14:47:52: Thread 0 has lock
14:47:52: Thread 1 about to lock
14:47:52: Thread 0 about to release lock
14:47:52: Thread 0 after release
14:47:52: Thread 1 has lock
14:47:52: Thread 0: finishing update
14:47:52: Thread 1 about to release lock
14:47:52: Thread 1 after release
14:47:52: Thread 1: finishing update
14:47:52: Testing update. Ending value is 2.
